In [1]:
import asyncio

from src.api import request_api_animes
from src.workers import worker_request
from src.workers import worker_process
from src.workers import worker_join

In [2]:
async def main(logger, mode=dict(), request_params=dict(), db='animedb.sqlite'):
    logger.info('Starting anime update')
    logger.info(f'Mode: {mode}')
    logger.info(f'Request params: {request_params}')

    queue_request = asyncio.Queue()
    queue_process = asyncio.Queue()
    queue_final   = asyncio.Queue()

    queue_request.put_nowait(['anime', 1, request_api_animes(request_params)])
    task_request = asyncio.create_task(worker_request(
        [queue_request, queue_process],
        logger = logger,
        sleep = 1
    ))
    task_process = asyncio.create_task(worker_process(
        [queue_request, queue_process, queue_final],
        dict(
            mode = mode,
            request_params = request_params,
            db = db
        ),
        logger = logger,
        sleep = 1
    ))
    task_join = asyncio.create_task(worker_join(
        queue_final,
        logger = logger,
        sleep = 2
    ))

    logger.info('Waiting for task_process')
    await asyncio.wait_for(task_process, 86400)
    logger.info('Stopping task_request...')
    task_request.cancel()
    logger.info('Mark task_done in queue_request...')
    queue_final.task_done() # last item should be marked to done
    logger.info('Waiting for queue_final...')
    await queue_final.join()
    logger.info('Stopping task_join...')
    task_join.cancel()
    return await queue_final.get()



In [3]:
import logging

def create_logger(name):
    formatter = logging.Formatter(
        fmt='%(asctime)s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )

    file_handler = logging.FileHandler('logger-work.log', mode='w')
    file_handler.setFormatter(formatter)

    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(file_handler)

    return logger

In [4]:
logger = create_logger('logger-work')

In [5]:
dicts = await main(
    mode = dict(
        type = 'newest',
    ),
    request_params = dict(
        order_by = 'start_date',
        start_date = '2023-06-01'
    ),
    logger = logger,
    db='animedb.sqlite'
)

In [7]:
# def get_request_params(mode, mode_kwargs):
#     modes_request = dict(
#         full_recreate = dict(page=1),
#         all = dict(page=1),
#         newest = dict(
#             order_by = 'start_date',
#             start_date = modes_kwargs.get('start_date', '2023-01-01'),
#             page=1
#         ),
#         scores = dict(
#             order_by = 'scored_by',
#             page=1
#         )
#     )

#     return modes_request.get('mode', dict(page=1))
#     request_params = get_request_params(mode, mode_kwargs)